## Trabajo Fin de Grado
### Gestor de Quirófanos
### Implementación de Algoritmo Particle Swarm para optimización del orden de las intervenciones quirúrgicas

#### Autor: Jesús García Armario

In [43]:
# Imports necesarios
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import sys
sys.path.append('../')
from Heuristicas.Utils import Quirofano, ActoQuirurgico
from Genético.utilsGenetico import cromosomaAleatorio, evaluar, validar, distance, cruce, mutacion

In [44]:
# Importamos el listado preprocesado de una especialidad para las pruebas
filename = '..\\..\\Datos\\Listados_Preprocesados\\ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx'
df = pd.read_excel(filename)
# Seleccionamos una muestra aleatoria de 100 pacientes
df = df.sample(n=100, random_state=1)
# Nos quedamos sólo con el NHC, Ponderación y duración
df = df[['NHC', 'PONDERACIÓN', 'DURACIÓN']]
# Sumamos a la duración 25 minutos por paciente para tener en cuenta el tiempo de preparación
df['DURACIÓN'] = df['DURACIÓN'] + 25
ventana = 30
# Dividimos la duración entre la ventana y redondeamos hacia arriba
df['DURACIÓN'] = df['DURACIÓN'].apply(lambda x: math.ceil(x/ventana))
# Creamos un set de actos quirúrgicos
actos_pendientes = list()
i = 0
for elemento in df.itertuples():
    actos_pendientes.append(ActoQuirurgico(i, elemento[3], elemento[1], elemento[2]))
    i += 1

In [45]:
# Creamos una población de 100 partículas válidas
def crear_poblacion(tiempos, quirofanos, dias, actos_pendientes, n):
    poblacion = []
    for i in range(n):
        poblacion.append({'cromosoma': cromosomaAleatorio(tiempos, quirofanos, dias, actos_pendientes), 'fitness': 0, 'velocidad': random.randint(2,n), 'personal_best': None, 'fitness_personal_best': 0})
    return poblacion

In [46]:

# Evaluamos a cada partícula
def evaluar_poblacion(poblacion, tiempos = 16, quirofanos = 3, dias = 5, actos_pendientes = actos_pendientes):
    for part in poblacion:
        part['fitness'] = evaluar(part['cromosoma'], tiempos, quirofanos, dias, actos_pendientes)[0]
    return poblacion

# Inicializamos la mejor partícula
def mejor_particula(poblacion):
    pop = sorted(poblacion, key=lambda k: k['fitness'])
    return pop[0]

# Inicializamos la personal best de cada partícula
def personal_best(poblacion):
    for part in poblacion:
        part['personal_best'] = part['cromosoma']
        part['fitness_personal_best'] = part['fitness']
    return poblacion

In [47]:
# Creamos la población inicial
poblacion = crear_poblacion(16, 3, 5, actos_pendientes, 100)
# Evaluamos la población
poblacion = evaluar_poblacion(poblacion)
# Inicializamos la mejor partícula
mejor = mejor_particula(poblacion)
# Inicializamos la personal best de cada partícula
poblacion = personal_best(poblacion)
# Inicializamos los parámetros
w = 0.9
c1 = 0.5
c2 = 0.3
mejor_global = mejor
# Inicializamos el contador de iteraciones
iteraciones = 0
# Inicializamos el contador de iteraciones sin mejora
iteraciones_sin_mejora = 0
# Inicializamos el número máximo de iteraciones sin mejora
max_iteraciones_sin_mejora = 100
# Inicializamos el número máximo de iteraciones
max_iteraciones = 1000

print('Mejor solución inicial: ', mejor_global)

# Iniciamos el bucle
while iteraciones < max_iteraciones and iteraciones_sin_mejora < max_iteraciones_sin_mejora:
    # Actualizamos la mejor partícula global
    if mejor['fitness'] < mejor_global['fitness']:
        mejor_global = mejor
        iteraciones_sin_mejora = 0
    else:
        iteraciones_sin_mejora += 1
    # Actualizamos las velocidades y posiciones
    for part in poblacion:
        # Actualizamos la velocidad
        part['velocidad'] = w * part['velocidad'] + c1 * random.random() * (mejor['fitness'] - part['fitness']) + c2 * random.random() * (mejor_global['fitness'] - part['fitness'])
        # Actualizamos la posición
        part['fitness'] = part['fitness'] + part['velocidad']
        # Si es negativa, volvemos al mejor valor personal
        if part['fitness'] < 0:
            part['fitness'] = part['fitness_personal_best']
            part['cromosoma'] = part['personal_best']
        else:
            # Realizamos la mutación
            part['cromosoma'] = mutacion(part['cromosoma'])
    # Seleccionamos los individuos a cruzar
    # Serán los mejores de la población
    poblacion = sorted(poblacion, key = lambda i: i['fitness'])
    # Cruzamos a dos individuos al azar
    individuos = random.choices(poblacion, k=2)
    hijo1, hijo2 = cruce(individuos[0]['cromosoma'], individuos[1]['cromosoma'])
    # Evaluamos a los nuevos individuos
    hijo1_fit= evaluar(hijo1, 16, 3, 5, actos_pendientes)[0]
    hijo2_fit= evaluar(hijo2, 16, 3, 5, actos_pendientes)[0]
    # Sustituimos los dos peores de la población por los nuevos individuos
    poblacion[-1]['cromosoma'] = hijo1
    poblacion[-1]['fitness'] = hijo1_fit
    poblacion[-2]['cromosoma'] = hijo2
    poblacion[-2]['fitness'] = hijo2_fit
    # Actualizamos la personal best de cada partícula
    for part in poblacion:
        if part['fitness'] < part['fitness_personal_best']:
            part['personal_best'] = part['cromosoma']
            part['fitness_personal_best'] = part['fitness']

    # Actualizamos la mejor partícula
    mejor = mejor_particula(poblacion)
    # Actualizamos el contador de iteraciones
    iteraciones += 1

# Mostramos el resultado
print('Mejor solución: ', mejor_global['cromosoma'])
print('Fitness: ', mejor_global['fitness'])
print('Iteraciones: ', iteraciones)
print('Evaluación del mejor: ', evaluar(mejor_global['cromosoma'], 16, 3, 5, actos_pendientes)[0])

Mejor solución inicial:  {'cromosoma': [48, 2, 11, 23, 37, 'A', 0, 92, 30, 74, 79, 44, 84, 'V', 22, 'A', 95, 85, 8, 54, 'A', 'B', 75, 34, 'V', 45, 'A', 41, 1, 89, 35, 'A', 57, 6, 40, 93, 'A', 'B', 82, 68, 24, 67, 53, 96, 5, 'A', 28, 19, 18, 43, 33, 29, 42, 'V', 27, 'A', 63, 60, 21, 13, 51, 59, 'V', 86, 'A', 'B', 88, 9, 69, 64, 'A', 32, 97, 'V', 76, 'A', 72, 10, 77, 15, 'A', 'B', 81, 47, 16, 'A', 87, 65, 46, 39, 'A', 80, 14, 26, 25, 12, 90, 70, 'A', 'B'], 'fitness': 1, 'velocidad': 81, 'personal_best': [48, 2, 11, 23, 37, 'A', 0, 92, 30, 74, 79, 44, 84, 'V', 22, 'A', 95, 85, 8, 54, 'A', 'B', 75, 34, 'V', 45, 'A', 41, 1, 89, 35, 'A', 57, 6, 40, 93, 'A', 'B', 82, 68, 24, 67, 53, 96, 5, 'A', 28, 19, 18, 43, 33, 29, 42, 'V', 27, 'A', 63, 60, 21, 13, 51, 59, 'V', 86, 'A', 'B', 88, 9, 69, 64, 'A', 32, 97, 'V', 76, 'A', 72, 10, 77, 15, 'A', 'B', 81, 47, 16, 'A', 87, 65, 46, 39, 'A', 80, 14, 26, 25, 12, 90, 70, 'A', 'B'], 'fitness_personal_best': 1}
Mejor solución:  [6, 86, 79, 51, 38, 69, 'A',